In [1]:
import pandas as pd
from pathlib import Path

crawl = pd.read_csv("../006_crawling_juznevesti.csv")
segments = pd.read_json('../014_filtered_df.json', lines=True)
segments["text"] = segments.asr_transcription
segments["kaldi_transcript"] = ""
segments["kaldi_words"] = ""
segments["kaldi_word_starts"] = ""
segments["kaldi_word_ends"] = ""

Constructing a `asr_results.json`:

we need a jsonl file with fields:
+ file
+ start
+ end
+ text

In [2]:
segments.file.values[-1][6:-4]

'fTfyE4ZdZqY'

In [4]:
from utils.matching import Matcher, load_segments
from tqdm.auto import tqdm
# test_hash = "58xZSVbpgkk"
# test_hash = 'fTfyE4ZdZqY'
for test_hash in tqdm(segments.file.unique()):
    try:
        test_hash = test_hash[6:-4]

        file = Path("../audio") / Path(test_hash +".wav")
        transcript = crawl[crawl.yt_hash==test_hash].transcript.values[0]

        def preprocess_gold(text: str) -> str:
            # Remove whitespace:
            text = " ".join(text.split())

            # Remove `JV:` prompts:
            text = text.replace("JV: ", "")
            
            # Lowercase:
            text = text.casefold()

            return text

        Path('sample.txt').write_text(preprocess_gold(transcript))
        matcher = Matcher(Path('sample.txt'))
        segments_subset = segments[segments.file.str.contains(test_hash)]
        with open('asr_results.json', 'w') as f:
            import json
            segments_subset["file"] = segments_subset.file.apply(lambda s: "../"+s)
            for i, seg in segments_subset[["file", "start", "end", "text"]].dropna().iterrows():
                f.write(json.dumps(seg.to_dict()) + '\n')
        asr_results = load_segments(Path('asr_results.json'))
        positions = matcher.match(asr_results)
        # matcher.print_debug(positions)
        positions = matcher.resegment_positions(positions)
        import shutil
        shutil.rmtree("./work", ignore_errors=True)
        %mkdir work
        %mkdir work/data
        matcher.make_kaldi_dir(positions, Path('work/data'), Path('.'))
        cmd ="""cut -f1 -d' ' work/data/segments | awk '{print $1,$1}' > work/data/utt2spk;
    cut -f1 -d' ' work/data/wav.scp | awk '{print $1,$1,"A"}' > work/data/reco2file_and_channel;
    cp work/data/utt2spk work/data/spk2utt;
    export LC_ALL=C;
    for f in work/data/* ; do
        sort -o $f $f
    done"""
        import os
        os.system(cmd)
        os.system("./utils/kaldi_align.sh")
        results = pd.read_csv("work/ali/ctm", sep="\s+", names=["reco", "A", "start", "duration", "word"])
        results["end"] = results.start + results.duration

        file_as_in_segments = file.__str__()[3:]
        c_file = segments.file == file_as_in_segments

        for i, row in segments[c_file].iterrows():
            s = row['start']
            e = row['end']

            current = results[(results.start >= s) & (results.end <= e)]
            kaldi_transcript = " ".join(current.word.values)
            kaldi_starts = current.start.values.tolist()
            kaldi_ends = current.end.values.tolist()
            kaldi_words = current.word.tolist()

            segments.loc[i, "kaldi_transcript"] = kaldi_transcript
            segments.loc[i, "kaldi_words"] = str(kaldi_words)
            segments.loc[i, "kaldi_word_starts"] = str(kaldi_starts)
            segments.loc[i, "kaldi_word_ends"] = str(kaldi_ends)
    except:
        continue

  0%|          | 0/289 [00:00<?, ?it/s]

/tmp/ipykernel_292481/1002024574.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segments_subset["file"] = segments_subset.file.apply(lambda s: "../"+s)


./steps/make_mfcc.sh --nj 2 --mfcc-config conf/mfcc_hires.conf data
utils/validate_data_dir.sh: Successfully validated data-directory data
./steps/make_mfcc.sh [info]: segments file exists: using that.
./steps/make_mfcc.sh: Succeeded creating MFCC features for data
./steps/compute_cmvn_stats.sh data
Succeeded creating CMVN stats for data
./steps/online/nnet2/extract_ivectors_online.sh --nj 2 data exp/nnet3/extractor ivectors


filter_scps.pl: warning: some input lines were output to multiple files [OK if splitting per utt] 
filter_scps.pl: warning: some input lines were output to multiple files [OK if splitting per utt] 


./steps/online/nnet2/extract_ivectors_online.sh: extracting iVectors
./steps/online/nnet2/extract_ivectors_online.sh: combining iVectors across jobs
./steps/online/nnet2/extract_ivectors_online.sh: done extracting (online) iVectors to ivectors using the extractor in exp/nnet3/extractor.


...

prepare_lang.sh: validating output directory
utils/validate_lang.pl lang
Checking existence of separator file
separator file lang/subword_separator.txt is empty or does not exist, deal in word case.
Checking lang/phones.txt ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> lang/phones.txt is OK

Checking words.txt: #0 ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> lang/words.txt is OK

Checking disjoint: silence.txt, nonsilence.txt, disambig.txt ...
--> silence.txt and nonsilence.txt are disjoint
--> silence.txt and disambig.txt are disjoint
--> disambig.txt and nonsilence.txt are disjoint
--> disjoint property is OK

Checking sumation: silence.txt, nonsilence.txt, disambig.txt ...
--> found no unexplainable phones in phones.txt

Checking lang/phones/context_indep.{txt, int, csl} ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only all

filter_scps.pl: warning: some input lines were output to multiple files [OK if splitting per utt] 
filter_scps.pl: warning: some input lines were output to multiple files [OK if splitting per utt] 


./steps/online/nnet2/extract_ivectors_online.sh: extracting iVectors
./steps/online/nnet2/extract_ivectors_online.sh: combining iVectors across jobs
./steps/online/nnet2/extract_ivectors_online.sh: done extracting (online) iVectors to ivectors using the extractor in exp/nnet3/extractor.
./utils/prepare_lang.sh dict <unk> tmp lang
Checking dict/silence_phones.txt ...
--> reading dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> dict/silence_phones.txt is OK

Checking dict/optional_silence.txt ...
--> reading dict/optional_silence.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> dict/optional_silence.txt is OK

Checking dict/nonsilence_phones.txt ...
--> reading dict/nonsilence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> dict/nonsilence_phones.txt is OK

Checking disjoint: silence

fstaddselfloops lang/phones/wdisambig_phones.int lang/phones/wdisambig_words.int 


prepare_lang.sh: validating output directory
utils/validate_lang.pl lang
Checking existence of separator file
separator file lang/subword_separator.txt is empty or does not exist, deal in word case.
Checking lang/phones.txt ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> lang/phones.txt is OK

Checking words.txt: #0 ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> lang/words.txt is OK

Checking disjoint: silence.txt, nonsilence.txt, disambig.txt ...
--> silence.txt and nonsilence.txt are disjoint
--> silence.txt and disambig.txt are disjoint
--> disambig.txt and nonsilence.txt are disjoint
--> disjoint property is OK

Checking sumation: silence.txt, nonsilence.txt, disambig.txt ...
--> found no unexplainable phones in phones.txt

Checking lang/phones/context_indep.{txt, int, csl} ...
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only all

In [5]:
segments.to_json("017_after_kaldi.jsonl", orient="records", lines=True)

In [6]:
segments

,file,start,end,asr_transcription,webpage_transcript,ratio,text,kaldi_transcript,kaldi_words,kaldi_word_starts,kaldi_word_ends
0,audio/58xZSVbpgkk.wav,12.0,31.0,nekoliko puta smo ponavljali da nišo od uvođen...,Nekoliko puta smo ponavljali da Niš od uvođenj...,0.757895,nekoliko puta smo ponavljali da nišo od uvođen...,nekoliko puta smo ponavljali da niš od uvođenj...,"['nekoliko', 'puta', 'smo', 'ponavljali', 'da'...","[12.47, 12.87, 13.06, 13.25, 13.69, 13.81, 14....","[12.870000000000001, 13.059999999999999, 13.25..."
1,audio/58xZSVbpgkk.wav,31.0,47.0,nikad više pa niti iskoristio niti će izgledat...,"- nikad više?’ Niti je iskoristio, niti će izg...",0.780000,nikad više pa niti iskoristio niti će izgledat...,nikad više niti je iskoristio niti će izgleda ...,"['nikad', 'više', 'niti', 'je', 'iskoristio', ...","[31.17, 31.46, 32.6, 32.83, 32.89, 33.54, 33.7...","[31.46, 31.810000000000002, 32.83, 32.89, 33.5..."
2,audio/58xZSVbpgkk.wav,47.0,62.5,dobrani i znate da e svaka stranka od polih pr...,Vi znate da svaka stranka otprilike predloži 2...,0.773333,dobrani i znate da e svaka stranka od polih pr...,vi znate da svaka stranka otprilike predloži d...,"['vi', 'znate', 'da', 'svaka', 'stranka', 'otp...","[47.73, 47.84, 48.06, 48.19, 48.49, 48.89, 49....","[47.839999999999996, 48.06, 48.190000000000005..."
3,audio/58xZSVbpgkk.wav,62.5,82.1,znači grad uopštenije organizvan i mi faktički...,"gradu! Znači, grad uopšte nije organizovan i m...",0.725275,znači grad uopštenije organizvan i mi faktički...,znači grad uopšte nije organizovan i mi faktič...,"['znači', 'grad', 'uopšte', 'nije', 'organizov...","[62.99, 63.28, 63.63, 63.96, 64.14, 65.05, 65....","[63.28, 63.63, 63.96, 64.14, 64.88, 65.17, 65...."
4,audio/58xZSVbpgkk.wav,82.1,98.4,pa iima ali ja kažem da bi trebao grada kažem ...,"ja jesam... Pa ima, ali ja kažem da bi trebalo...",0.604167,pa iima ali ja kažem da bi trebao grada kažem ...,pa ima ali ja kažem da bi trebalo grad da kaže...,"['pa', 'ima', 'ali', 'ja', 'kažem', 'da', 'bi'...","[82.1, 82.33, 82.92, 83.1, 83.25, 83.58, 83.66...","[82.33, 82.92, 83.10000000000001, 83.25, 83.58..."
...,...,...,...,...,...,...,...,...,...,...,...
16528,audio/fTfyE4ZdZqY.wav,1139.2,1144.6,poziciju povrenika ako hoćete da se kandidute ...,bio najobičniji politikanski trik. Saša Jankov...,0.800000,poziciju povrenika ako hoćete da se kandidute ...,poziciju poverenika ako hoćete da se kandiduje...,"['poziciju', 'poverenika', 'ako', 'hoćete', 'd...","[1139.52, 1140.11, 1140.99, 1141.26, 1141.69, ...","[1140.11, 1140.9599999999998, 1141.26, 1141.69..."
16529,audio/fTfyE4ZdZqY.wav,1144.6,1170.2,slične su poruke bile upućivane i tadašnjem om...,drugoj poziciji kada je najavio političku kand...,0.608696,slične su poruke bile upućivane i tadašnjem om...,slične su poruke bile upućivae i saši janković...,"['slične', 'su', 'poruke', 'bile', 'upućivae',...","[1145.37, 1145.81, 1146.15, 1146.65, 1146.85, ...","[1145.81, 1146.12, 1146.65, 1146.8500000000001..."
16530,audio/fTfyE4ZdZqY.wav,1170.2,1188.3,nije realno da li to je op razmišljam dakle sv...,potez. Utoliko pre što je predsednica Skupštin...,0.703297,nije realno da li to je op razmišljam dakle sv...,nije realno da o toj opciji ne razmišljam sve ...,"['nije', 'realno', 'da', 'o', 'toj', 'opciji',...","[1170.31, 1170.56, 1170.9, 1171.01, 1171.05, 1...","[1170.56, 1170.87, 1171.01, 1171.05, 1171.22, ..."
16531,audio/fTfyE4ZdZqY.wav,1188.3,1202.4,izvrštaj povjerenika pa da onda vidimo da li j...,je to zakonska obaveza i poslovnička obaveza S...,0.750000,izvrštaj povjerenika pa da onda vidimo da li j...,izveštaj poverenika pa da onda vidimo da li je...,"['izveštaj', 'poverenika', 'pa', 'da', 'onda',...","[1188.56, 1189.0, 1189.6, 1189.73, 1189.87, 11...","[1189.0, 1189.55, 1189.73, 1189.8700000000001,..."
